# Importing Libraries, "importing" data sets and setting storage

In a working situation there would be 1000s of schemes, domains and safe names that could be imported here. I have just taken the example of a few banks for simplicity.

In [233]:
from difflib import SequenceMatcher

#########
# Look Up Matricies - obviously replacable with a csv etc, that has more entires
#########
scheme = ["https://", "mailto://", "ftp://"]
top_level_domain = [".com", ".fi", ".co.uk", ".edu", ".org"]

#########
# Look Up dintionary - again obviously replacable with a csv etc, that has more entires
#########
safe_name = {"DanskeBank": "danskebank", "OP": "op", "Nordea": "nordea", "Aktia": "aktia"}

#########
#Storage system
#########

strange_features_index = {}
gateway_characters_index = {}
url_info = {}
url_sim_info = {}

# Creating functions to then impliment in my final function

The functions are in three general themes
1. Seperating the first free url (https://arxiv.org/pdf/1510.06501.pdf) - this can be changed to seperate any part of the url, I was simply following the beginning of the article above

2. Checking a url for similarities to a known one

3. Checking a url for strange features

In [136]:
########
#Remove the first free url from a url
########

# Removes the scheme, by deleting it from the url
def scheme_splicer(url):
    #Remove scheme
    for sche in scheme:
        if (url.find(sche) != -1):
            url = url[len(sche):]
            return url

# Removes the end, after the domain by deleting any values after the url
def top_level_domain_splicer(url):
    for tld in top_level_domain:
        if (url.find(tld) != -1):
            url = url[:url.find(tld)]
            return url
        #### This will cut out things that start wuth .fi like finance

# Combines the previous two
def url_splicer(url):
    if scheme_splicer(url) != None:
        url_no_scheme = scheme_splicer(url)
    else:
        url_no_scheme = url
        # as there is a finite list of possible scheme this should only happen if here is not a scheme in the first place
        # Or a typo
    if top_level_domain_splicer(url_no_scheme) != None:
        free_url = top_level_domain_splicer(url_no_scheme)
    else:
        free_url = url_no_scheme
        #This again, should work as there is also a finite numner of top level domains
        #If there is no top level domain caught there was none in the first place
        #Although a more extensive list is needed than the one I have above
    return free_url

In [215]:
#############
#Name Reference
#############

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
    # This finds the similar characters in strings and trandforms them into ratios
    # Critisism - this is position based, could be improved with caracter and position comparason

#printing a string with the official website if so
def name_checker(url):
    url = url_splicer(url)
    for website in safe_name:
        #Find if website has the same first url
        if ((similar(url, safe_name[website]) == 1) == True):
            return "Offical Url (first free): {web}".format(web = website)
        #if not ture then how similar are the name

In [100]:
###########
#Checking the url for strnage features
###########

#This function prints off a dictionary of the number of each feature
def strange_featrues(url):
    #Safe characters for urls (check these)
    strange_features_index['dot'] = url.count('.')
    strange_features_index['dollar'] = url.count('$')
    strange_features_index['plus'] = url.count('+')
    strange_features_index['apostrophe'] = url.count('\'')
    strange_features_index['dash'] = url.count('-')
    strange_features_index['semicolon'] = url.count(';')
    strange_features_index['colon'] = url.count(':')
    strange_features_index['comma'] = url.count(',')
    strange_features_index['slash'] = url.count('/')
    strange_features_index['question_mark'] = url.count('?')
    strange_features_index['at'] = url.count('@')
    strange_features_index['equals'] = url.count('=')
    strange_features_index['and'] = url.count('&')
    return strange_features_index
    
    
#This function prints off a dictionary of the number of each gateway feature
def gateway_featrues(url):
    #Gateway modifiable characters, these are places that the phisher could take you away from the original website
    gateway_characters_index['o_curley_bracket'] = url.count("{")
    gateway_characters_index['c_curley_bracket']  = url.count("}")
    gateway_characters_index['o_square_bracket']  = url.count("[")
    gateway_characters_index['c_square_bracket']  = url.count("]")
    gateway_characters_index['tilde'] = url.count("~")
    gateway_characters_index['^'] = url.count("^")
    gateway_characters_index['line'] = url.count("|")
    return gateway_characters_index

# My Final Function

This function takes all of the other functions information and complies them into a dictionary labled with the original url ready for later diagnosis. I have only gathered esthetic features here, it would be nice to also get the age of the url, amounts of views (possition in a google search) etc. but the method would be the same.

In [246]:
#######
#Final Function
#######
def url_checker(url):
    # Adds the url to the dictionary as the key
    url_info[url]={}
    safe_names = safe_name
    #Update the domain 
    if (name_checker(url) != None):
        url_info[url]["Domain_Analysis"] = name_checker(url)
        url_info[url]["Similarity_Index"] = "NA"
    else: 
        url_info[url]["Domain_Analysis"] = "Unofficial"
        safe_names = safe_name # to protect my original data, just in case
        for name in safe_names:
            url_sim_info[name] = similar(url, safe_names[name])
            url_info[url]["Similarity_Index"] = url_sim_info
            
    #Check Unusual Features
    url_info[url]["Length"] = len(url)
    url_info[url]["Strange_Features"] = strange_featrues(url)
    url_info[url]["Gateway_Features"] = gateway_featrues(url)
    
    return url_info

# Example Function and Result Presentation

Although having a presentation in the code itself is not that useful, I though that seeing the dictionary as an output was dificult to analyse immediately and thus difficult to see what my code produces. Although in reality you would then run more functions on this url so would not present it at this stage and present it in a much better way - in terms of the task it adds clarity. Dictionaries are an easily idexable storgae stsyem in the code with easy updates (these are the main reason I used them), however they are not easly to read by themselves.

In [250]:
def print_results(url):
    print("""{url}
    \t Domain Analysis: {da}
    \t Similarity Index: {si}
    \t Length: {l}
    \t Strange Features:
    Dollar: {d} \t Plus: {p} \t Apostrophe: {ap} \t Dash: {d2} \t Semicolon: {s} \t Colon: {sc} \t Comma: {co} \t Slash: {sl} \t Question Mark: {qm} \t At: {at} \t Equals: {eq} \t And: {a}
    \t Gateway Features:
    Open Curley Bracket: {ocb} \t Closed Curley Bracket: {ccb} \t Open Square Brackets: {osb} \t Closed Square Brackets: {csb} \t Tilde: {td} \t Hat: {hat} \t Line: {line}
    """.format(si = url_info[url]["Similarity_Index"], url = url, l = url_info[url]["Length"], d = url_info[url]["Strange_Features"]["dollar"], p = url_info[url]["Strange_Features"]["plus"], ap = url_info[url]["Strange_Features"]["apostrophe"], d2 = url_info[url]["Strange_Features"]["dash"], s = url_info[url]["Strange_Features"]["semicolon"], sc = url_info[url]["Strange_Features"]["colon"], co = url_info[url]["Strange_Features"]["comma"], sl = url_info[url]["Strange_Features"]["slash"], qm = url_info[url]["Strange_Features"]["question_mark"], at = url_info[url]["Strange_Features"]["at"], eq = url_info[url]["Strange_Features"]["equals"], a = url_info[url]["Strange_Features"]["and"], ocb = url_info[url]["Gateway_Features"]["o_curley_bracket"], ccb = url_info[url]["Gateway_Features"]["c_curley_bracket"], osb = url_info[url]["Gateway_Features"]["o_square_bracket"], csb = url_info[url]["Gateway_Features"]["c_square_bracket"], td = url_info[url]["Gateway_Features"]["tilde"], hat = url_info[url]["Gateway_Features"]["^"], line = url_info[url]["Gateway_Features"]["line"], da = url_info[url]["Domain_Analysis"]))


# Example

In [249]:
lst = ["https://op.fi/this.is.not.a.scam", "https://o.p.fi/this.is.a.#scam", "ftp://n.o#r@d-e$a.fi"]

for url in lst:
    url_checker(url)
    print_results(url)

https://op.fi/this.is.not.a.scam
    	 Domain Analysis: Offical Url (first free): OP
    	 Similarity Index: NA
    	 Length: 32
    	 Strange Features:
    Dollar: 0 	 Plus: 0 	 Apostrophe: 0 	 Dash: 0 	 Semicolon: 0 	 Colon: 1 	 Comma: 0 	 Slash: 3 	 Question Mark: 0 	 At: 0 	 Equals: 0 	 And: 0
    	 Gateway Features:
    Open Curley Bracket: 0 	 Closed Curley Bracket: 0 	 Open Square Brackets: 0 	 Closed Square Brackets: 0 	 Tilde: 0 	 Hat: 0 	 Line: 0
    
https://o.p.fi/this.is.a.#scam
    	 Domain Analysis: Unofficial
    	 Similarity Index: {'DanskeBank': 0.1, 'OP': 0.0625, 'Nordea': 0.1111111111111111, 'Aktia': 0.17142857142857143}
    	 Length: 30
    	 Strange Features:
    Dollar: 0 	 Plus: 0 	 Apostrophe: 0 	 Dash: 0 	 Semicolon: 0 	 Colon: 1 	 Comma: 0 	 Slash: 3 	 Question Mark: 0 	 At: 0 	 Equals: 0 	 And: 0
    	 Gateway Features:
    Open Curley Bracket: 0 	 Closed Curley Bracket: 0 	 Open Square Brackets: 0 	 Closed Square Brackets: 0 	 Tilde: 0 	 Hat: 0 	 Line: 0
  